In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pickle

In [3]:
import pandas
import sqlite3

## Some Filenames

In [4]:
source_file = "data/test_pages.pkl"
bill_links_file = "data/bill_links_all.pkl"
keywords_file = "data/bill_page_keywords.pkl"

## Load Pages

In [5]:
with open(source_file, 'rb') as f1:
    pages = pickle.load(f1)

In [6]:
len(pages)

2100

In [7]:
bill_info = pandas.DataFrame([{k : page[k] for k in ['session', 'house', 'bill']} for page in pages])

In [8]:
bill_info.head()

,bill,house,session
0,1,H,2015E4
1,2,H,2015E4
2,3,H,2015E4
3,4,H,2015E4
4,5,H,2015E4


In [9]:
pages[0]['page'][:200]

b'<!doctype html>\n<html>\n<head>\n\t<meta name="description" content="The Official Site of the North Carolina General Assembly.">\n\t<meta name="keywords" content="NCGA, North Carolina General Assembly, offi'

In [10]:
soups = [bs(page['page'], 'html.parser') for page in pages]

## Make Soup and Extract Content of Interest

### Titles

In [362]:
titles = [soup.find('div', {"id":"title"}).text.strip() for soup in soups]

In [363]:
titles[:15]

['House Rules for Fourth Extra Session.',
 'Adjourn 2016 Fourth Extra Session Sine Die.',
 'Regulatory Reform Act of 2016.',
 'Terminate Agreement for Tolling of I-77.',
 'Municipal Broadband Service Area.',
 'Independent State CIO.',
 'Study Validity of I-77 Tolling Contract.',
 'Prohibit Discriminatory Profiling.',
 'Dog Breeding Stds./Law Enforcement Tools.',
 'Clarify Funding Eligibility/Charter Schools.',
 'DOT/Moratorium on Reductions.',
 'School Calendar Flex/CC.',
 'Class Size Requirement Changes.',
 'Greene County Funds.',
 'Honor State Employees and Teachers.']

### Bill Content Links

In [13]:
soup = soups[0]

In [14]:
tables = soup.find_all("table")
print(len(tables))

18


In [15]:
content_table = [t for t in tables if t.text.strip().startswith("View Available Bill Summaries")][-1]

In [16]:
content_table

<table cellspacing="0" style="width: 100%; margin: 0px; padding: 0px;">
<tr><td colspan="2" style="background-color: #ced0d2; padding: 0px; text-align: center; font-weight: bold;border-bottom: 1px solid #555;border-top: 1px solid #555;"><a href="/gascripts/billsummaries/billsummaries.pl?Session=2015E4&amp;BillID=H1">View Available Bill Summaries</a> </td></tr>
<tr class="sectionBG1">
<th class="tableHeader" style="border-bottom: 1px solid #555; padding-left: 5px;">Bill Text</th><th class="tableHeader" nowrap="" style="border-bottom: 1px solid #555;">Fiscal Note</th>
</tr>
<tr>
<td nowrap="" style="border-bottom: 1px dotted #DCDCDC; padding-left:4px; padding-bottom:4px; vertical-align: top;"><a href="/Sessions/2015E4/Bills/House/PDF/H1v0.pdf">Filed</a> <span style="font: 9px Arial;">[<a href="/Sessions/2015E4/Bills/House/HTML/H1v0.html">HTML</a>]</span></td>
<td style="border-bottom: 1px dotted #DCDCDC; padding-left:4px; padding-bottom:4px; vertical-align: top;"><a href=""></a><div styl

In [17]:
for row in content_table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) > 1:
        label = cols[0].text
        links = cols[0].find_all('a')
        pdf_link = links[0]['href']
        html_link = links[1]['href']
        print('{} - {} - {}'.format(label, pdf_link, html_link))

Filed [HTML] - /Sessions/2015E4/Bills/House/PDF/H1v0.pdf - /Sessions/2015E4/Bills/House/HTML/H1v0.html
Edition 1 [HTML] - /Sessions/2015E4/Bills/House/PDF/H1v1.pdf - /Sessions/2015E4/Bills/House/HTML/H1v1.html
Edition 2 [HTML] - /Sessions/2015E4/Bills/House/PDF/H1v2.pdf - /Sessions/2015E4/Bills/House/HTML/H1v2.html
Adopted [HTML] - /Sessions/2015E4/Bills/House/PDF/H1v3.pdf - /Sessions/2015E4/Bills/House/HTML/H1v3.html


In [34]:
def extractBillLinks(soup, session, house, bill):
    tables = soup.find_all("table")
    content_table = [t for t in tables if t.text.strip().startswith("View Available Bill Summaries")][-1]
    bill_text_links = []
    for row in content_table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 1:
            try:
                label = cols[0].text.strip('[HTML]').strip().encode('utf8').replace(b'\xc2\xa0', b' ')
                links = cols[0].find_all('a')
                pdf_link = links[0]['href']
                html_link = links[1]['href']
                bill_text_links.append({'session' : session, 'house' : house, 'bill' : bill,
                                        'label' : label, 'pdf' : pdf_link, 'html' : html_link})
            except IndexError:
                pass
    return(bill_text_links)


def extractBillFiledLinks(soup):
    tables = soup.find_all("table")
    content_table = [t for t in tables if t.text.strip().startswith("View Available Bill Summaries")][-1]
    filed_links = {}
    for row in content_table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 1:
            label = cols[0].text.strip('[HTML]').strip().encode('utf8').replace(b'\xc2\xa0', b' ')
            if label in [b'Filed']:
                links = cols[0].find_all('a')
                pdf_link = links[0]['href']
                html_link = links[1]['href']
                filed_links = {'label' : label, 'pdf' : pdf_link, 'html' : html_link}
                break
    return(filed_links)


def extractBillFinalLinks(soup):
    tables = soup.find_all("table")
    content_table = [t for t in tables if t.text.strip().startswith("View Available Bill Summaries")][-1]
    adopted_links = {}
    for row in content_table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 1:
            label = cols[0].text.strip('[HTML]').strip().encode('utf8').replace(b'\xc2\xa0', b' ')
            if label in [b'Adopted', b'Ratified']:
                links = cols[0].find_all('a')
                pdf_link = links[0]['href']
                html_link = links[1]['href']
                adopted_links = {'label' : label, 'pdf' : pdf_link, 'html' : html_link}
    return(adopted_links)

In [35]:
extractBillLinks(soup, bill_info.session[0], bill_info.house[0], bill_info.bill[0])

[{'bill': 1,
  'house': 'H',
  'html': '/Sessions/2015E4/Bills/House/HTML/H1v0.html',
  'label': b'Filed',
  'pdf': '/Sessions/2015E4/Bills/House/PDF/H1v0.pdf',
  'session': '2015E4'},
 {'bill': 1,
  'house': 'H',
  'html': '/Sessions/2015E4/Bills/House/HTML/H1v1.html',
  'label': b'Edition 1',
  'pdf': '/Sessions/2015E4/Bills/House/PDF/H1v1.pdf',
  'session': '2015E4'},
 {'bill': 1,
  'house': 'H',
  'html': '/Sessions/2015E4/Bills/House/HTML/H1v2.html',
  'label': b'Edition 2',
  'pdf': '/Sessions/2015E4/Bills/House/PDF/H1v2.pdf',
  'session': '2015E4'},
 {'bill': 1,
  'house': 'H',
  'html': '/Sessions/2015E4/Bills/House/HTML/H1v3.html',
  'label': b'Adopted',
  'pdf': '/Sessions/2015E4/Bills/House/PDF/H1v3.pdf',
  'session': '2015E4'}]

In [20]:
extractBillFiledLinks(soup)

{'html': '/Sessions/2015E4/Bills/House/HTML/H1v0.html',
 'label': b'Filed',
 'pdf': '/Sessions/2015E4/Bills/House/PDF/H1v0.pdf'}

In [21]:
extractBillFinalLinks(soup)

{'html': '/Sessions/2015E4/Bills/House/HTML/H1v3.html',
 'label': b'Adopted',
 'pdf': '/Sessions/2015E4/Bills/House/PDF/H1v3.pdf'}

#### All Links

In [36]:
bill_links = [extractBillLinks(soup, bi.session, bi.house, bi.bill) \
              for (bi, soup) in zip(bill_info.itertuples(), soups)]

In [39]:
bill_links = [item for sublist in bill_links for item in sublist]
bill_links = pandas.DataFrame(bill_links)

In [41]:
bill_links = bill_links[['session', 'house', 'bill', 'label', 'html', 'pdf']]

In [43]:
bill_links.head()

,session,house,bill,label,html,pdf
0,2015E4,H,1,b'Filed',/Sessions/2015E4/Bills/House/HTML/H1v0.html,/Sessions/2015E4/Bills/House/PDF/H1v0.pdf
1,2015E4,H,1,b'Edition 1',/Sessions/2015E4/Bills/House/HTML/H1v1.html,/Sessions/2015E4/Bills/House/PDF/H1v1.pdf
2,2015E4,H,1,b'Edition 2',/Sessions/2015E4/Bills/House/HTML/H1v2.html,/Sessions/2015E4/Bills/House/PDF/H1v2.pdf
3,2015E4,H,1,b'Adopted',/Sessions/2015E4/Bills/House/HTML/H1v3.html,/Sessions/2015E4/Bills/House/PDF/H1v3.pdf
4,2015E4,H,2,b'Filed',/Sessions/2015E4/Bills/House/HTML/H2v0.html,/Sessions/2015E4/Bills/House/PDF/H2v0.pdf


In [44]:
bill_links.tail()

,session,house,bill,label,html,pdf
6291,2015,S,902,b'Resolution 2016-22',/Sessions/2015/Bills/Senate/HTML/S902v2.html,/Sessions/2015/Bills/Senate/PDF/S902v2.pdf
6292,2015,S,903,b'Filed',/Sessions/2015/Bills/Senate/HTML/S903v0.html,/Sessions/2015/Bills/Senate/PDF/S903v0.pdf
6293,2015,S,903,b'Edition 1',/Sessions/2015/Bills/Senate/HTML/S903v1.html,/Sessions/2015/Bills/Senate/PDF/S903v1.pdf
6294,2015,S,903,b'Edition 2',/Sessions/2015/Bills/Senate/HTML/S903v2.html,/Sessions/2015/Bills/Senate/PDF/S903v2.pdf
6295,2015,S,903,b'Resolution 2016-23',/Sessions/2015/Bills/Senate/HTML/S903v3.html,/Sessions/2015/Bills/Senate/PDF/S903v3.pdf


In [45]:
with open(bill_links_file, 'wb') as f1:
    pickle.dump(bill_links, f1)

#### Filed Links Only

In [294]:
filed_bill_links = pandas.DataFrame([extractBillFiledLinks(soup) for soup in soups])

In [295]:
filed_bill_links = filed_bill_links.join(bill_info, how="left")

In [296]:
filed_bill_links = filed_bill_links[['session', 'house', 'bill', 'label', 'html', 'pdf']]

In [297]:
filed_bill_links.head()

,session,house,bill,label,html,pdf
0,2015E4,H,1,b'Filed',/Sessions/2015E4/Bills/House/HTML/H1v0.html,/Sessions/2015E4/Bills/House/PDF/H1v0.pdf
1,2015E4,H,2,b'Filed',/Sessions/2015E4/Bills/House/HTML/H2v0.html,/Sessions/2015E4/Bills/House/PDF/H2v0.pdf
2,2015E4,H,3,b'Filed',/Sessions/2015E4/Bills/House/HTML/H3v0.html,/Sessions/2015E4/Bills/House/PDF/H3v0.pdf
3,2015E4,H,4,b'Filed',/Sessions/2015E4/Bills/House/HTML/H4v0.html,/Sessions/2015E4/Bills/House/PDF/H4v0.pdf
4,2015E4,H,5,b'Filed',/Sessions/2015E4/Bills/House/HTML/H5v0.html,/Sessions/2015E4/Bills/House/PDF/H5v0.pdf


In [298]:
filed_bill_links.shape

(2100, 6)

#### Final Links Only

In [299]:
final_bill_links = pandas.DataFrame([extractBillFinalLinks(soup) for soup in soups])
final_bill_links.dropna(inplace=True)

In [300]:
final_bill_links = final_bill_links.join(bill_info, how='left')

In [301]:
final_bill_links = final_bill_links[['session', 'house', 'bill', 'label', 'html', 'pdf']]

In [302]:
final_bill_links.head()

,session,house,bill,label,html,pdf
0,2015E4,H,1,b'Adopted',/Sessions/2015E4/Bills/House/HTML/H1v3.html,/Sessions/2015E4/Bills/House/PDF/H1v3.pdf
16,2015E4,H,17,b'Ratified',/Sessions/2015E4/Bills/House/HTML/H17v6.html,/Sessions/2015E4/Bills/House/PDF/H17v6.pdf
24,2015E4,S,1,b'Adopted',/Sessions/2015E4/Bills/Senate/HTML/S1v2.html,/Sessions/2015E4/Bills/Senate/PDF/S1v2.pdf
27,2015E4,S,4,b'Ratified',/Sessions/2015E4/Bills/Senate/HTML/S4v6.html,/Sessions/2015E4/Bills/Senate/PDF/S4v6.pdf
31,2015E3,H,1,b'Adopted',/Sessions/2015E3/Bills/House/HTML/H1v2.html,/Sessions/2015E3/Bills/House/PDF/H1v2.pdf


In [303]:
final_bill_links.shape

(459, 6)

### Keywords

In [307]:
ds = soup.find_all('tr')
len(ds)

80

In [316]:
trs = [tr for tr in ds if len(tr.find_all('th')) > 0]
len(trs)

11

In [325]:
ths = [tr for tr in trs if tr.find('th').text.lower().startswith('keyword')]

In [331]:
keywords = [w.strip() for w in  ths[0].div.text.split(',')]

In [333]:
keywords

['APPOINTMENTS',
 'BOARDS',
 'CHAPTERED',
 'COMMERCE',
 'COMMISSIONS',
 'COUNCIL OF STATE',
 'COUNTIES',
 'COURT OF APPEALS',
 'COURTS',
 'ELECTIONS',
 'ELECTIONS BOARDS',
 'ELECTIONS',
 'STATE BOARD OF',
 'ETHICS',
 'ETHICS COMN.',
 'GENERAL ASSEMBLY',
 'GOVERNOR',
 'INDUSTRIAL COMN.',
 'LOBBYING',
 'LOCAL GOVERNMENT',
 'MEMBERSHIP',
 'POLITICAL ACTIVITY',
 'POLITICAL PARTIES',
 'PRESENTED',
 'PRESIDENT PRO TEMPORE',
 'PUBLIC',
 'PUBLIC OFFICIALS',
 'RATIFIED',
 'REDISTRICTING',
 'SECRETARY OF STATE',
 'SPEAKER',
 'SUPREME COURT',
 'TERM LENGTHS & LIMITS']

In [335]:
def extractKeywords(soup):
    ds = soup.find_all('tr')
    trs = [tr for tr in ds if len(tr.find_all('th')) > 0]
    ths = [tr for tr in trs if tr.find('th').text.lower().startswith('keyword')]
    keywords = [w.strip() for w in  ths[0].div.text.split(',')]
    return(keywords)

In [338]:
extractKeywords(soup)[:10]

['APPOINTMENTS',
 'BOARDS',
 'CHAPTERED',
 'COMMERCE',
 'COMMISSIONS',
 'COUNCIL OF STATE',
 'COUNTIES',
 'COURT OF APPEALS',
 'COURTS',
 'ELECTIONS']

In [348]:
keywords = [{'keywords' : extractKeywords(soup)} for soup in soups]

In [350]:
keywords = pandas.DataFrame.join(bill_info, pandas.DataFrame(keywords))

In [353]:
keywords.head()

,bill,house,session,keywords
0,1,H,2015E4,"[ADOPTED, GENERAL ASSEMBLY, RESOLUTIONS, SIMPL..."
1,2,H,2015E4,"[ADJOURNMENT, GENERAL ASSEMBLY, RESOLUTIONS, J..."
2,3,H,2015E4,"[ADMINISTRATION DEPT., ADMINISTRATIVE CODE, AD..."
3,4,H,2015E4,"[BRIDGES, CONTRACTS, COUNTIES, INFRASTRUCTURE,..."
4,5,H,2015E4,"[COUNTIES, EDGECOMBE COUNTY, INFORMATION TECHN..."


In [355]:
with open(keywords_file, 'wb') as f1:
    pickle.dump(keywords, f1)

## General Analysis Ideas

2. Get keywords associated with bills
  1. Get filed bill texts
  1. Can we get groups based on keywords - text?
3. determine if bill ever passed
  1. get passed bill texts
  1. Can we get groups based on passed/accepted - text?

### First Pass Storage

1. Pages go to mongo? hdfs? (who cares??)
2. general bill info to (My)SQL
  1. just the session and house and ID info
  2. also a sql assigned key? or will this just
3. link info to a 'links' table
  1. each line is a new bill state
  2. so 'session - house - label - html_link - pdf_link'
4. bill content table
  1. raw text
  2. hdfs?  mongo? (need key info...)
4. events table (future)